In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
features = pd.read_csv('../datasets/UNSW-NB15_features.csv')
features = features.replace(to_replace='srcip', value='ipv4_src')
features = features.replace(to_replace='sport', value='port_src')
features = features.replace(to_replace='dstip', value='ipv4_dst')
features = features.replace(to_replace='dsport', value='port_dst')
names = np.array(features['Name'])
real = pd.read_csv('../datasets/UNSW-NB15_1.csv', names=names)

/home/jozef/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (1,3,47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
real[['ipv4_src', 'port_src', 'ipv4_dst', 'port_dst']] = real[['ipv4_src', 'port_src', 'ipv4_dst', 'port_dst']].astype(str)

In [4]:
real.shape

(700001, 49)

In [5]:
pd.set_option('display.max_columns', None)

In [6]:
real.head()

,ipv4_src,port_src,ipv4_dst,port_dst,proto,state,dur,sbytes,dbytes,sttl,dttl,sloss,dloss,service,Sload,Dload,Spkts,Dpkts,swin,dwin,stcpb,dtcpb,smeansz,dmeansz,trans_depth,res_bdy_len,Sjit,Djit,Stime,Ltime,Sintpkt,Dintpkt,tcprtt,synack,ackdat,is_sm_ips_ports,ct_state_ttl,ct_flw_http_mthd,is_ftp_login,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,attack_cat,Label
0,59.166.0.0,1390,149.171.126.6,53,udp,CON,0.001055,132,164,31,29,0,0,dns,500473.93750,621800.93750,2,2,0,0,0,0,66,82,0,0,0.00000,0.000000,1421927414,1421927414,0.017,0.013000,0.0,0.0,0.0,0,0,0,0,0,3,7,1,3,1,1,1,NaN,0
1,59.166.0.0,33661,149.171.126.9,1024,udp,CON,0.036133,528,304,31,29,0,0,-,87676.08594,50480.17188,4,4,0,0,0,0,132,76,0,0,9.89101,10.682733,1421927414,1421927414,7.005,7.564333,0.0,0.0,0.0,0,0,0,0,0,2,4,2,3,1,1,2,NaN,0
2,59.166.0.6,1464,149.171.126.7,53,udp,CON,0.001119,146,178,31,29,0,0,dns,521894.53130,636282.37500,2,2,0,0,0,0,73,89,0,0,0.00000,0.000000,1421927414,1421927414,0.017,0.013000,0.0,0.0,0.0,0,0,0,0,0,12,8,1,2,2,1,1,NaN,0
3,59.166.0.5,3593,149.171.126.5,53,udp,CON,0.001209,132,164,31,29,0,0,dns,436724.56250,542597.18750,2,2,0,0,0,0,66,82,0,0,0.00000,0.000000,1421927414,1421927414,0.043,0.014000,0.0,0.0,0.0,0,0,0,0,0,6,9,1,1,1,1,1,NaN,0
4,59.166.0.3,49664,149.171.126.0,53,udp,CON,0.001169,146,178,31,29,0,0,dns,499572.25000,609067.56250,2,2,0,0,0,0,73,89,0,0,0.00000,0.000000,1421927414,1421927414,0.005,0.003000,0.0,0.0,0.0,0,0,0,0,0,7,9,1,1,1,1,1,NaN,0


In [7]:
means_names = ['Accuracy','Sensitivity','Specificity','Precision']
means = pd.DataFrame(columns=means_names)
means

,Accuracy,Sensitivity,Specificity,Precision


In [18]:
flows = pd.read_csv('../results/flows_dump_part1700-1702.txt', delimiter=',')
flows['real_label'] = 0
flows.head()

,ipv4_src,port_src,ipv4_dst,port_dst,proto,bytes_src,bytes_dst,packets_src,packets_dst,srv_dst_count,dst_count,category,probability,real_label
0,59.166.0.7,10346,149.171.126.1,7804,6,1848,33516,28,30,1,6,Normal,3.49%,0
1,59.166.0.7,41714,149.171.126.8,39575,6,2838,14546,36,36,1,3,Normal,1.80%,0
2,59.166.0.2,41241,149.171.126.7,53,17,160,192,2,2,1,3,Normal,1.29%,0
3,59.166.0.2,13294,149.171.126.3,80,6,1144,962,8,10,1,8,Normal,6.24%,0
4,59.166.0.2,14354,149.171.126.3,6881,6,1764,1896,16,18,7,8,Normal,0.09%,0


In [19]:
flows[['ipv4_src', 'port_src', 'ipv4_dst', 'port_dst']] = flows[['ipv4_src', 'port_src', 'ipv4_dst', 'port_dst']].astype(str)

In [20]:
def get_real_labels(ipv4_src, port_src, ipv4_dst, port_dst):
    #print ipv4_src, port_src, ipv4_dst, port_dst
    tmp = real['Label'].loc[(real['ipv4_src'] == ipv4_src) & 
               (real['port_src'] == port_src) &
               (real['ipv4_dst'] == ipv4_dst) &
               (real['port_dst'] == port_dst)]
    #print 'First:', tmp.values
    if tmp.values.size == 0:
        tmp = real['Label'].loc[(real['ipv4_src'] == ipv4_dst) & 
               (real['port_src'] == port_dst) &
               (real['ipv4_dst'] == ipv4_src) &
               (real['port_dst'] == port_src)]
        #print 'Second:', tmp.values
    return tmp.values[0]
    
flows['real_label'] = flows.apply(lambda row: get_real_labels(row['ipv4_src'], row['port_src'], row['ipv4_dst'], row['port_dst']), axis=1)
flows.head()

,ipv4_src,port_src,ipv4_dst,port_dst,proto,bytes_src,bytes_dst,packets_src,packets_dst,srv_dst_count,dst_count,category,probability,real_label
0,59.166.0.7,10346,149.171.126.1,7804,6,1848,33516,28,30,1,6,Normal,3.49%,0
1,59.166.0.7,41714,149.171.126.8,39575,6,2838,14546,36,36,1,3,Normal,1.80%,0
2,59.166.0.2,41241,149.171.126.7,53,17,160,192,2,2,1,3,Normal,1.29%,0
3,59.166.0.2,13294,149.171.126.3,80,6,1144,962,8,10,1,8,Normal,6.24%,0
4,59.166.0.2,14354,149.171.126.3,6881,6,1764,1896,16,18,7,8,Normal,0.09%,0


In [21]:
gr_data = flows[['ipv4_src', 'category']]
gr_data['count'] = 1
gr_data = gr_data.groupby(['ipv4_src', 'category']).count()
gr_data

/home/jozef/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


count
ipv4_src       category       
149.171.126.17 Attack        1
149.171.126.7  Normal        1
175.45.176.1   Attack        1
               Normal        1
175.45.176.2   Attack        2
               Normal        1
59.166.0.0     Attack        1
               Normal        1
59.166.0.1     Attack        1
59.166.0.2     Normal        8
59.166.0.4     Normal        3
59.166.0.5     Attack        1
               Normal       10
59.166.0.7     Normal        4

In [22]:
evaluation = flows[['ipv4_src', 'category', 'real_label']]
evaluation['ipv4_src'] = evaluation.apply(lambda row: 'Malicious user' if row['real_label'] == 1 else 'Normal user', axis=1)
evaluation = evaluation[['ipv4_src', 'category']]
evaluation['count'] = 1
# evaluation = evaluation.replace(to_replace=r'(^59.*$)', value='Normal user', regex=True)
# evaluation = evaluation.replace(to_replace=r'(^175.*$)', value='Malicious user', regex=True)
evaluation = evaluation.groupby(['ipv4_src', 'category']).count()
evaluation

/home/jozef/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


count
ipv4_src       category       
Malicious user Attack        3
               Normal        1
Normal user    Attack        4
               Normal       28

In [23]:
e = evaluation.unstack(fill_value=0)
e

count       
category       Attack Normal
ipv4_src                    
Malicious user      3      1
Normal user         4     28

In [24]:
try:
    TP = e.loc['Malicious user'].loc['count'].loc['Attack']
except:
    TP = 0
try:
    TN = e.loc['Normal user'].loc['count'].loc['Normal']
except:
    TN = 0
try:
    FP = e.loc['Normal user'].loc['count'].loc['Attack']
except:
    FP = 0
try:
    FN = e.loc['Malicious user'].loc['count'].loc['Normal']
except:
    FN = 0
print 'TP:', TP
print 'TN:', TN
print 'FP:', FP
print 'FN:', FN

TP: 3
TN: 28
FP: 4
FN: 1


In [25]:
try:
    acc = float(TP + TN)/float(TP + TN + FP + FN)
except:
    acc = 0
print ('Accuracy is %.2f%%' % (acc*100))
try:
    sen = float(TP)/float(TP + FN)
except:
    sen = 0
print ('Sensitivity is %.2f%%' % (sen*100))
try:
    spe = float(TN)/float(TN + FP)
except:
    spe = 0
print ('Specificity is %.2f%%' % (spe*100))
try:
    pre = float(TP)/float(TP + FP)
except:
    pre = 0
print ('Precision is %.2f%%' % (pre*100))

Accuracy is 86.11%
Sensitivity is 75.00%
Specificity is 87.50%
Precision is 42.86%


In [26]:
flows.shape

(36, 14)

In [27]:
packets_count = flows[['packets_src','packets_dst']].sum()
packets_count.sum()

2584

In [191]:
df = pd.DataFrame([[acc, sen, spe, pre]], columns=means_names)
means = means.append(df)

In [192]:
means

,Accuracy,Sensitivity,Specificity,Precision
0,0.750000,1.000000,0.727273,0.250000
0,0.807692,1.000000,0.791667,0.285714
0,0.761905,0.666667,0.777778,0.333333
0,0.741935,0.428571,0.833333,0.428571
0,0.769231,0.800000,0.761905,0.444444
0,0.780000,0.750000,0.782609,0.230769
0,0.702128,0.600000,0.714286,0.200000
0,0.977273,0.888889,0.983740,0.800000
0,0.975410,1.000000,0.973684,0.727273


In [193]:
means['Accuracy'].mean()

0.8072859495123974

In [194]:
means['Sensitivity'].mean()

0.7926807760141092

In [195]:
means['Specificity'].mean()

0.8162526360908716

In [196]:
means['Precision'].mean()

0.4111228277894945

## Notes and unused commands

In [24]:
try:
    f1_score = 2*(float(pre*sen)/float(pre+sen))
except:
    f1_score = 0
print ('F1 score is %.2f%%' % (f1_score*100))

F1 score is 75.00%
